# 02 - Data Preparation

## Introduction

To improve inventory management at Corporación Favorita, we need to prepare the data in a way that supports accurate, product-level sales forecasting. Our business goal is to reduce **understocking and overstocking** by anticipating future demand at each store for each product family. 

To support this, we will prepare:
- Historical **sales data** (store-product-day level)
- **Promotions**, **transactions**, and **holiday indicators**
- External context like **oil prices** (economic signal for demand)

This combined dataset will form the foundation for modeling product-level demand and improving stocking decisions.

---

### Model Selection: Why We Use Prophet

We are using **Prophet** as our forecasting model because it aligns directly with the business requirements:

- It captures **seasonality and holiday effects**, which heavily impact retail sales.
- It handles **missing dates** and **irregular demand** — common in real retail operations.
- It allows us to model **each store-product family separately**, enabling tailored inventory strategies.
- It produces **interpretable forecasts** that help non-technical teams understand sales patterns.

These features make Prophet well-suited for solving Corporación Favorita’s stock management challenges.


---


### Objectives of This Notebook

This notebook prepares a clean, feature-rich dataset ready for time series forecasting with Prophet. Our objectives:

- **Filter and align data**:
  - Keep relevant columns from sales, holidays, oil, and transactions datasets.
  - Merge all sources at the daily store-family level.

- **Engineer key features**:
  - Lag features (e.g. `sales_lag_7`, `sales_lag_14`)
  - Rolling averages (e.g. 7-day, 14-day smoothed sales)
  - Flags: `onpromotion`, `is_holiday`

- **Format for Prophet**:
  - Rename `date` to `ds`, `sales` to `y`
  - Include all relevant regressors to enrich the model



## 1. Import & Load Data

In [108]:
# Import necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)

# Ingore warnings
import warnings
warnings.filterwarnings('ignore')

# Load datasets
sales_data = pd.read_csv('../data/train.csv')
stores_data = pd.read_csv('../data/stores.csv')
holidays_events_data = pd.read_csv('../data/holidays_events.csv')
oil_data = pd.read_csv('../data/oil.csv')
transactions_data = pd.read_csv('../data/transactions.csv')

## 2. Inspect and Clean Data

### 2.1 Convert date columns

In [109]:
# Convert date columns to datetime format
# Ensures consistency across all datasets for accurate merging
sales_data['date'] = pd.to_datetime(sales_data['date'])
transactions_data['date'] = pd.to_datetime(transactions_data['date'])
holidays_events_data['date'] = pd.to_datetime(holidays_events_data['date'])
oil_data['date'] = pd.to_datetime(oil_data['date'])


## 2.2 Inspect Mising Values

In [110]:
# Check for missing values in external datasets
# Helps identify gaps that need to be filled before merging
print("Missing values in oil_data:")
print(oil_data.isna().sum())

# Check for gaps in oil_data dates
oil_date_range = pd.date_range(start=oil_data['date'].min(), end=oil_data['date'].max())
missing_oil_dates = set(oil_date_range) - set(oil_data['date'].unique())
print(f"\nNumber of missing dates in oil_data: {len(missing_oil_dates)}")
if missing_oil_dates:
    print("Missing oil_data dates:", sorted(missing_oil_dates))
else:
    print("No missing dates in oil_data.")

print("\nMissing values in transactions_data:")
print(transactions_data.isna().sum())

# Check for gaps in dates in transactions_data
date_range = pd.date_range(start=transactions_data['date'].min(), end=transactions_data['date'].max())
missing_dates = set(date_range) - set(transactions_data['date'].unique())
print(f"\nNumber of missing dates in transactions_data: {len(missing_dates)}")
if missing_dates:
    print("Missing dates:", sorted(missing_dates))
else:
    print("No missing dates in transactions_data.")


Missing values in oil_data:
date           0
dcoilwtico    43
dtype: int64

Number of missing dates in oil_data: 486
Missing oil_data dates: [Timestamp('2013-01-05 00:00:00'), Timestamp('2013-01-06 00:00:00'), Timestamp('2013-01-12 00:00:00'), Timestamp('2013-01-13 00:00:00'), Timestamp('2013-01-19 00:00:00'), Timestamp('2013-01-20 00:00:00'), Timestamp('2013-01-26 00:00:00'), Timestamp('2013-01-27 00:00:00'), Timestamp('2013-02-02 00:00:00'), Timestamp('2013-02-03 00:00:00'), Timestamp('2013-02-09 00:00:00'), Timestamp('2013-02-10 00:00:00'), Timestamp('2013-02-16 00:00:00'), Timestamp('2013-02-17 00:00:00'), Timestamp('2013-02-23 00:00:00'), Timestamp('2013-02-24 00:00:00'), Timestamp('2013-03-02 00:00:00'), Timestamp('2013-03-03 00:00:00'), Timestamp('2013-03-09 00:00:00'), Timestamp('2013-03-10 00:00:00'), Timestamp('2013-03-16 00:00:00'), Timestamp('2013-03-17 00:00:00'), Timestamp('2013-03-23 00:00:00'), Timestamp('2013-03-24 00:00:00'), Timestamp('2013-03-30 00:00:00'), Timestam

### 2.3 Fill Missing Dates in Oil & Transsactions Data

In [111]:
# Create a full daily date range matching sales_data
full_dates = pd.date_range(start=sales_data['date'].min(), end=sales_data['date'].max())

# --- Clean oil_data ---
oil_data = oil_data.set_index('date').reindex(full_dates).rename_axis('date').reset_index()
oil_data['dcoilwtico'] = oil_data['dcoilwtico'].fillna(method='ffill')

# --- Clean transactions_data ---
# Expand transactions_data to cover all store_nbr + full_dates combinations
store_ids = transactions_data['store_nbr'].unique()
full_index = pd.MultiIndex.from_product([full_dates, store_ids], names=['date', 'store_nbr'])
transactions_data = transactions_data.set_index(['date', 'store_nbr']).reindex(full_index).reset_index()
transactions_data['transactions'] = transactions_data['transactions'].fillna(0)


> Filling missing dates in oil_data and transactions_data ensures we have a complete daily timeline — essential for time series modeling. Without this, we risk misaligned data, gaps in regressors, and invalid training in models like Prophet. Forward-filling oil prices and zero-filling transactions helps preserve business logic while avoiding bias or errors.

### 2.4 Fill missing values
It prevents `NaN`s from leaking into the model features. We avoid `NaN`s because most ML models (like Prophet) can’t handle them, and they break forecasting and analysis.

In [112]:
# Fill missing oil prices in both directions
oil_data['dcoilwtico'] = oil_data['dcoilwtico'].fillna(method='ffill').fillna(method='bfill')

# Fill missing transactions data with 0 
transactions_data['transactions'] = transactions_data['transactions'].fillna(0)

## 2.5 Inspect Holidays

In [113]:
# Check unique holiday types and locales
print("Holiday Types:", holidays_events_data['type'].unique())
print("Locales:", holidays_events_data['locale'].unique())
print("Transferred values:", holidays_events_data['transferred'].value_counts())

# Preview structure
holidays_events_data.head()

Holiday Types: ['Holiday' 'Transfer' 'Additional' 'Bridge' 'Work Day' 'Event']
Locales: ['Local' 'Regional' 'National']
Transferred values: transferred
False    338
True      12
Name: count, dtype: int64


,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


## 2.6 Filter Holidays

In [114]:
# Filter only usable holidays (exclude transferred)
usable_holidays = holidays_events_data[holidays_events_data['transferred'] == False].copy()

# Keep only relevant columns
usable_holidays = usable_holidays[['date', 'type', 'locale', 'locale_name']]


## 2.7 Inspect Stores 

In [115]:
# Check for unique cities and states
print("Cities:", stores_data['city'].nunique())
print("States:", stores_data['state'].nunique())

# Preview store metadata
stores_data.head()

Cities: 22
States: 16


,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


## 3. Merge Data

### 3.1 Base Merge

In [116]:
# --- Ensure date columns are in datetime format ---
sales_data['date'] = pd.to_datetime(sales_data['date'])
transactions_data['date'] = pd.to_datetime(transactions_data['date'])
holidays_events_data['date'] = pd.to_datetime(holidays_events_data['date'])
oil_data['date'] = pd.to_datetime(oil_data['date'])

# --- Merge sales with transactions ---
sales_base = pd.merge(
    sales_data,
    transactions_data,
    on=['date', 'store_nbr'],
    how='left'
)

# --- Merge with oil prices ---
sales_base = pd.merge(
    sales_base,
    oil_data,
    on='date',
    how='left'
)

# --- Merge with store metadata ---
sales_base = pd.merge(
    sales_base,
    stores_data,
    on='store_nbr',
    how='left'
)

# --- Add holiday flag (National holidays only) ---
usable_holidays = holidays_events_data[holidays_events_data['transferred'] == False]
national_holiday_dates = usable_holidays[usable_holidays['locale'] == 'National']['date'].unique()
sales_base['is_holiday'] = sales_base['date'].isin(national_holiday_dates).astype(int)

# --- Final check ---
sales_base = sales_base.sort_values(by=['store_nbr', 'family', 'date']).reset_index(drop=True)

# Preview
sales_base.head()


,id,date,store_nbr,family,sales,onpromotion,transactions,dcoilwtico,city,state,type,cluster,is_holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,0.0,93.14,Quito,Pichincha,D,13,1
1,1782,2013-01-02,1,AUTOMOTIVE,2.0,0,2111.0,93.14,Quito,Pichincha,D,13,0
2,3564,2013-01-03,1,AUTOMOTIVE,3.0,0,1833.0,92.97,Quito,Pichincha,D,13,0
3,5346,2013-01-04,1,AUTOMOTIVE,3.0,0,1863.0,93.12,Quito,Pichincha,D,13,0
4,7128,2013-01-05,1,AUTOMOTIVE,5.0,0,1509.0,93.12,Quito,Pichincha,D,13,1


## 4. Feature Engineering

### 4.1 Rolling Averages
Rolling averages help smooth out daily fluctuations in sales and capture short-term momentum

In [117]:
# Sort by store, family, and date to prepare for rolling
sales_base = sales_base.sort_values(by=['store_nbr', 'family', 'date'])

# Compute 7-day and 14-day rolling averages of sales
sales_base['rolling_avg_7'] = (
    sales_base
    .groupby(['store_nbr', 'family'])['sales']
    .transform(lambda x: x.rolling(window=7, min_periods=1).mean())
)

sales_base['rolling_avg_14'] = (
    sales_base
    .groupby(['store_nbr', 'family'])['sales']
    .transform(lambda x: x.rolling(window=14, min_periods=1).mean())
)


### 4.2 Lagged Sales
Lagged sales features give the model a temporal memory

In [118]:
# Create lagged features for past sales
sales_base['sales_lag_1'] = (
    sales_base
    .groupby(['store_nbr', 'family'])['sales']
    .shift(1)
)

sales_base['sales_lag_7'] = (
    sales_base
    .groupby(['store_nbr', 'family'])['sales']
    .shift(7)
)

sales_base['sales_lag_14'] = (
    sales_base
    .groupby(['store_nbr', 'family'])['sales']
    .shift(14)
)


> These features, combined with rolling averages, form the backbone of most demand-aware inventory forecasting models.

## 5. Export Preprocessed Data

In [119]:
import os

# Ensure the output directory exists
os.makedirs('../data/processed', exist_ok=True)

# Save the final processed DataFrame to a CSV
sales_base.to_csv('../data/processed/inventory_prepared.csv', index=False)
